<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 2.d</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Catchment Delineation</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='images\ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

# Table of contents
1. [Learning objectives](#learningobs)
2. [Introduction](#introduction)
3. [Mosaic DEM tiles](#mosaic)
4. [Reproject and subset the DEM](#reproject)
5. [Convert to PCRaster format](#convert)
6. [Calculate flow direction](#flowdir)
7. [Delineate streams](#streams)
8. [Catchment delineation](#catchment)
9. [Automate Stream and Catchment Delineations](#automate)

# 1. Learning objectives<a name="learningobs"></a>

- To define different OGC API's
- To access WCS layers
- To show WCS metadata
- To download WCS layers
- To visualise the downloaded layer

# 2. Introduction<a name="introduction"></a>

In this lesson we're going to delineate streams and a catchment boundary from a DEM. In the Data folder you can find 4 SRTM DEM tiles in GeoTIFF format and shapefile with a rough bounding box of the study area.

The steps are:

1. Mosaic SRTM DEM tiles
2. Reproject and clip the DEM
3. Calculate the flow direction
4. Delineate the streams
5. Define the outlet
6. Delineate the catchment

Steps 1 and 2 are done with GDAL in Python. Then we convert the output to PCRaster format and continue with PCRaster operations in Python.

[Here is a playlist](https://youtube.com/playlist?list=PLeuKJkIxCDj05rwD3Fun7v7rpVRg7c_kE) with the theory and how to do this in QGIS.

Let's start with creating a mosaic from the SRTM tiles in the section.

# 3. Mosaic DEM tiles<a name="mosaic"></a>
The first step is to mosaic the DEM tiles into one raster layer.

Data used in this tutorial are in the `2d_Data` folder. Let's set the path first.

In [ ]:
import os
os.chdir('./2d_data')
print(os.getcwd())

Now we can make a list of all GeoTIFF files in a folder. We use the <code>glob</code> module to find all files matching the <code>.tif</code> extension.

In [ ]:
import glob
from osgeo import gdal

inputFiles = glob.glob('*.tif')
print(inputFiles)

As you see we've created a list of all the GeoTIFF files in the folder.
Now we're going to use that list to create a mosaic in a virtual file (<code>.vrt</code>). A virtual file is very efficient in this case, because it doesn't recreate the wholde dataset, but just describes how the tiles are connected. We use <code>gdal.BuildVRT</code> for that, with the default settings. It only needs the output filename (<code>mosaic.vrt</code>) and the list of input files. We need to add <code>FlushCache()</code> to write the layer to disk.

In [ ]:
mosaic = gdal.BuildVRT('mosaic.vrt',inputFiles)
mosaic.FlushCache()

You can now open the <code>mosaic.vrt</code> layer in QGIS to check the result.

The next step is to reproject and subset the mosaic so we have a smaller area to continue with.

# 4. Reproject and subset the DEM <a name="reproject"></a>
In this section we'll clip the mosaic to a smaller bounding box to reduce calculation times. In reality you'll have to estimate the approximate boundary. In the same step we reproject the DEM to UTM Zone 32 N / WGS-84 (EPSG: 32632).

We'll create a function to reproject and clip the mosaic using <code>gdal.Warp</code>. With <code>gdal.WarpOptions</code> you can choose different options that are described [here](https://gdal.org/python/osgeo.gdal-module.html#WarpOptions).

In this case we'll use:

* <code>cutlineDSName=shapefile</code> to define the shapefile that we use for clipping. In our case <code>boundingbox.shp</code>
* <code>cropToCutline=True</code> to use the cutline extent for the output bounds
* <code>format='GTIFF'</code> to indicate that the output format is GeoTIFF
* <code>dstSRS=projection</code> to define the output target projection, which is in our case EPSG: 32632.
* <code>xRes,yRes</code> are the x and y pixel dimensions which we set both to 30 m.

Then <code>gdal.Warp</code> uses <code>srcDSOrSrcDSTab</code> to define the input raster and <code>destNameOrDestDS</code> to define the output raster. We also add the <code>options</code> from <code>gdal.WarpOptions</code>.

We write this in a function to keep it readable:

In [ ]:
from osgeo import gdal
def reprojectAndClip(inputraster,outputraster,projection,shapefile,resolution):
    options = gdal.WarpOptions(cutlineDSName=shapefile,
                           cropToCutline=True,
                           format='GTIFF',
                           dstSRS=projection,
                           xRes=resolution,
                           yRes=resolution)
    outimage=gdal.Warp(srcDSOrSrcDSTab=inputraster,
                           destNameOrDestDS=outputraster,
                           options=options)
Mosaic = 'mosaic.vrt'
Polygon = 'boundingbox.shp'
EPSG = 'EPSG:32632'
spatialResolution = 30
DEMSubset = 'DEMsubset.tif'
reprojectAndClip(Mosaic,DEMSubset,EPSG,Polygon,spatialResolution)

Check the result in QGIS.

The rest of the workflow will be done with PCRaster. So in the next section we're going to convert the DEM subset to the PCRaster format using GDAL in Python.

# 5. Convert to PCRaster format <a name="convert"></a>
The next steps will be done using PCRaster. In this section we're going to convert the <code>DEMSubset.tif</code> file to PCRaster format.
The DEM is continuous raster so we need to convert it to a PCRaster scalar map. We can use this function:

In [ ]:
#Import gdal
from osgeo import gdal, gdalconst

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Open existing dataset
    src_ds = gdal.Open( src_filename )
    
    #GDAL Translate
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None
    
ConvertToPCRaster("DEMsubset.tif","dem.map",gdalconst.GDT_Float32,"VS_SCALAR")

Now we can use the DEM in PCRaster. Let's read the DEM map and visualise it

In [ ]:
from pcraster import *
DEM = readmap('dem.map')
plot(DEM)

The next step is to calculate the flow direction.

# 6. Calculate flow direction <a name="flowdir"></a>
In the stream and catchment delineation procedure we need to first remove the pits in the DEM. Pits are pixels surrounded by only higher pixels. A catchment can have only one pit, the outlet, so the other pits have to be removed by a procedure called "fill sinks".
In PCRaster the <code>lddcreate</code> operation will both fill the DEM and derive the flow direction. The <code>lddcreate</code> operation needs the DEM as input and has 4 arguments to control thresholds for the filling algorithm. Details can be found in the [PCRaster documention](https://pcraster.geo.uu.nl/pcraster/4.3.0/documentation/pcraster_manual/sphinx/op_lddcreate.html).

Here we want to remove all pits, so we set the thresholds to a very high number, 1e31.

Be patient when you run the code below, it will take time.

In [ ]:
FlowDirection = lddcreate(DEM,1e31,1e31,1e31,1e31)
report(FlowDirection,'flowdir.map')
aguila(FlowDirection)

Check the result by zooming in. The flow direction map has the data type ldd. Aguila recognises that and visualises the flow direction more intuitively by connected lines. Each catchment has a pit, indicated by a square.

*What is the general direction of the flow?*

*Why do we have so many pits at the sides of the map?*

Although we can proceed with the flow direction map, you might want to create a DEM that is filled. You can use the <code>lddcreatedem</code> operation for that, which has the same arguments as <code>lddcreate</code> but results in a hydrologically corrected DEM.

In [ ]:
DEMFilled = lddcreatedem(DEM,1e31,1e31,1e31,1e31)

In the field below visualise the result and calculate the difference between <code>DEM</code> and <code>DEMFilled</code> using map algebra.

# 7. Delineate streams <a name="streams"></a>
In this step we're going to delineate the streams. There are two methods that we're going to apply:
* Using Strahler orders
* Using flow accumulation

## 7.1 Strahler method
Let's start with deriving the Strahler orders using the <code>streamorder</code> operation.
The <code>streamorder</code> operation needs the flow direction map created in the previous section as an input. A detailed description of the <code>streamorder</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_streamorder.html).

In [ ]:
StrahlerOrders = streamorder(FlowDirection)
aguila(StrahlerOrders)

Change the legend in Aguila to a ramp from yellow to blue, so the higher orders are blue, like rivers.

The data type of the Strahler order map is ordinal, starting at 1. Now we need to determine after which Strahler order we consider the the flow big enough to call it a river. We do that through calibration. Let's calculate maps with Strahler order 1 to the maximum and compare them with OpenStreetMap.
First we determine the maximum Strahler order by using the <code>mapmaximum</code> operation. The <code>mapmaximum</code> operation is a global operation. More info can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_mapmaximum.html).

In [ ]:
MaximumStrahlerOrder = mapmaximum(StrahlerOrders)

Visualise the result with Aguila

The problem is that the result is a PCRaster map with for each pixel the maximum value of the map. In order to iterate over the Strahler orders we need to get the value. We can use the <code>cellvalue</code> operation. The <code>cellvalue</code> operation needs the raster for which it has the give the cell value as input as well as a row and column index number. Here all cells are the same, so we can simply use 0 for the index, refering to the first row and first column of the input raster.

In [ ]:
MaximumStrahlerOrderTuple = cellvalue(MaximumStrahlerOrder,0,0)
print(MaximumStrahlerOrderTuple)

The <code>cellvalue</code> operation returns a tuple with two elements: the first is the cell value, the second is a boolean value which shows whether the first element, is valid or not. If the second element is False, the cell contains a missing value. So if we want to have the value we need to take the first element of the tuple.

In [ ]:
MaximumStrahlerOrderValue = MaximumStrahlerOrderTuple[0]
print(MaximumStrahlerOrderValue)

Can you write a generic Python function below that reads a map and returns the cell value, so we can re-use it when we need it?

Now we can loop over the Strahler orders and save each map larger or equal to that order.

In [ ]:
for order in range (1,MaximumStrahlerOrderValue + 1):
    Stream = ifthen(StrahlerOrders >= order, boolean(1))
    report(Stream,'stream'+str(order)+'.map')

Visualise the different results and choose which best matches with OpenStreetMap or Google Satellite. You can load the layers in QGIS to do this.

## 7.2 Flow accumulation method
In order to estimate where the river is we can also use the flow accumulation method. In PCRaster that's done with the [accuflux](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_accuflux.html) operation. Accuflux will accumulate material (second argument) over the flow direction. Here we choose 1 for all cells as material.

In [ ]:
FlowAccumulation = accuflux(FlowDirection,1)

Visualise the result with Aguila. Change the legend to shifted logarithmic, because of the large range of values.

In the same way as with the Strahler method we need to determine the minimum flow accumulation that we consider a river. We do this again by comparing boolean maps with a reference map. Let's start wit 9000 pixels.

In [ ]:
RiverFlow = ifthen(FlowAccumulation > 9000, boolean(1))

Try this for different values and check how well it matches with for example OpenStreetMap in QGIS.

After calibration, write the one that fits best to disk with `report`.

# 8. Catchment Delineation <a name="catchment"></a>
In this section we're going to delineate the catchment of an outlet and all catchments in the DEM.

## 8.1 Delineate the catchment of a specific outlet

First we need to identify an outlet on the delineated stream of the previous step. You can use the result from the Strahler order method or the flow accumulation method for finding the pixels that are part of the river. You can use QGIS or Aguila to find the coordinate. In Aguila you can do that using the crosshair tool.

You can use the fields below to load the river map from the previous section, visualise with Aguila and find the coordinates of a pixel on the river map that we want to use as outlet for the catchment that we're going to delineate.

Let's use 288880.648,5675880.258 as coordinates of the outlet.
PCRaster comes with a tool <code>col2map</code>, which reads a textfile in the format:

`x y id`

and converts it to a PCRaster map. You can run the tool from the commandline, but here we're going to wrap it into a Python function. More info about the `col2map` tool can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/app_col2map.html)

The tool has arguments for the data type and needs a clone map. In our case the outlet will be a nominal raster with one pixel that has an id number. For the clone we can use the river map.

In [ ]:
def col2map(x,y,id,datatype,clone):
    with open('location.txt', 'w') as f:
        f.write(str(x) + ' ' + str(y) + ' ' + str(id))
    cmd = 'col2map -{0} location.txt location.map --clone {1}'.format(datatype,clone)
    print(cmd)
    os.system(cmd)
    Map = readmap('location.map')
    return Map

x = 288880.648
y = 5675880.258
id = 1
datatype = 'N'
clone = 'stream8.map'
Outlet = col2map(x,y,id,datatype,clone)
aguila(Outlet,river)

The function creates a text file. We wrote a string with "x y id" to the file. Then the command was constructed as a string and we run the command string with <code>os.system</code>.

We have also printed the command string after running the script. This command could be typed at the command prompt.

Now we have the outlet defined in our delineated river, we can delineate the catchment that contributes to that outlet using the <code>catchment</code> operation. The <code>catchment</code> operation needs the flow direction map and the outlet map as input. More information about the <code>catchment</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_catchment.html).

In [ ]:
RurCatchment = catchment(FlowDirection,Outlet)

## 8.2 Delineate all catchments in the DEM

That's the catchment of a specific outlet. If we want to automatically derive all catchments in the raster we need to find the pits first, which can be considered as outlets. This can be done with the <code>pit</code> operation. More information about the <code>pit</code> operation can be found in the [PCRaster documentation](https://pcraster.geo.uu.nl/pcraster/latest/documentation/pcraster_manual/sphinx/op_pit.html). It only needs the flow direction map as input.

In [ ]:
outlets = pit(FlowDirection)

Visualise the result with Aguila.

*Where are the pits located?*

*How many are there?*

Now we can use those outlets in the <code>catchment</code> operation to derive all catchments in the raster. Write the code below and visualise the result with Aguila.

*What do you see?*

*Why are there so many catchments at the boundary of the area?*

In the next section we're going to combine all code into one script to automate the process as much as possible.

# 9. Automate stream and catchment delineation <a name="automate"></a>
We can also automate the procedure of delineating streams and the catchment of a specific outlet by using the code of the previous sections. Obviously we can't automate the calibration of the stream delineation.

Try to combine all code of the previous sections first by yourself. Write it in functions to make it readable. You can use the example code below to help you out.

In [ ]:
import os, glob
from pcraster import *
from osgeo import gdal, gdalconst

def mosaic(inputpattern,outputmosaic):
    InputFiles = glob.glob(inputpattern)
    mosaic = gdal.BuildVRT(outputmosaic,InputFiles)
    mosaic.FlushCache()
    
def reprojectAndClip(inputraster,outputraster,projection,shapefile,resolution):
    options = gdal.WarpOptions(cutlineDSName=shapefile,
                           cropToCutline=True,
                           format='GTIFF',
                           dstSRS=projection,
                           xRes=resolution,
                           yRes=resolution)
    outimage=gdal.Warp(srcDSOrSrcDSTab=inputraster,
                           destNameOrDestDS=outputraster,
                           options=options)

def ConvertToPCRaster(src_filename,dst_filename,ot,VS):
    #Open existing dataset
    src_ds = gdal.Open( src_filename )
    
    #GDAL Translate
    dst_ds = gdal.Translate(dst_filename, src_ds, format='PCRaster', outputType=ot, metadataOptions=VS)
    
    #Properly close the datasets to flush to disk
    dst_ds = None
    src_ds = None
    
def CalculateFlowDirection(DEMFile):
    DEM = readmap(DEMFile)
    FlowDirectionMap = lddcreate(DEM,1e31,1e31,1e31,1e31)
    return FlowDirectionMap

def StreamDelineation(FlowDirectionMap,Threshold):
    StrahlerOrders = streamorder(FlowDirectionMap)
    Stream = ifthen(StrahlerOrders >= Threshold, boolean(1))
    return Stream
    
def col2map(x,y,clone):
    with open('location.txt', 'w') as f:
        f.write(str(x) + ' ' + str(y) + ' 1')
    cmd = 'col2map -N location.txt location.map --clone {0}'.format(clone)
    os.system(cmd)
    Map = readmap('location.map')
    return Map
    

# Define inputs and settings
# Note that outputs of previous runs need to be removed to avoid errors
# os.chdir('../PCRasterCatchmentDelineation/')
TileExtension = '*.tif'
MosaicOutput = 'mosaic.vrt'
BoundaryPolygon = 'boundingbox.shp'
OutputProjection = 'EPSG:32632'
OutputSpatialResolution = 30.0
DEMSubsetOutput = 'DEMsubset.tif'
PCRasterDEMOutput = 'dem.map'
FlowDirectionOutput = 'flowdir.map'
StrahlerOrderThreshold = 8
OutletX = 288880.648
OutletY = 5675880.258
clone = PCRasterDEMOutput


# Apply stream and catchment delineation workflow
print('Creating mosaic...')
mosaic(TileExtension,MosaicOutput)
print('Done!')

print('Reprojecting and clipping...')
reprojectAndClip(MosaicOutput,DEMSubsetOutput,OutputProjection,BoundaryPolygon,OutputSpatialResolution)
print('Done')

print('Converting to PCRaster format...')
ConvertToPCRaster(DEMSubsetOutput,PCRasterDEMOutput,gdalconst.GDT_Float32,"VS_SCALAR")
print('Done!')

print('Calculating flow direction...')
setclone(clone)
FlowDirection = CalculateFlowDirection(PCRasterDEMOutput)
print('Done!')

print('Delineating channels...')
River = StreamDelineation(FlowDirection,StrahlerOrderThreshold)
print('Done')

print('Delineating the catchment...')
Outlet = col2map(OutletX,OutletY,clone)
CatchmentArea = catchment(FlowDirection,Outlet)
print('Done')

#Visualise what you need
aguila(CatchmentArea)
aguila(FlowDirection)
aguila(River)

#Report what you need
report(CatchmentArea,'catchment.map')
report(FlowDirection,'flowdir.map')
report(River,'channels.map')